In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
file = 'final.csv'
df = pd.read_csv(file)
df = pd.DataFrame(df)
df

,Год,Направление,Уровень безработицы,Уровень инфляции,Вступительный балл,Пол,Образовательные специальные потребности,Дневная/вечерняя форма обучения,Предыдущее образование (оценка),Перемещенный,Возраст при зачислении,Международный статус,ВВП,Количество абитуриентов
0,2022.0,0.0,10.8,1.4,127.3,1,0,1,122.0,1,20,0,1.74,215
1,2023.0,1.0,13.9,-0.3,142.5,1,0,1,160.0,1,19,0,0.79,252
2,2024.0,2.0,10.8,1.4,124.8,1,0,1,122.0,1,19,0,1.74,226
3,2022.0,3.0,9.4,-0.8,119.6,0,0,1,122.0,1,20,0,-3.12,331
4,2023.0,4.0,13.9,-0.3,141.5,0,0,0,100.0,0,45,0,0.79,215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,2022.0,3.0,15.5,2.8,122.2,1,0,1,125.0,0,19,0,-4.06,331
4420,2023.0,3.0,11.1,0.6,119.0,0,0,1,120.0,1,18,1,2.02,331
4421,2024.0,6.0,13.9,-0.3,149.5,0,0,1,154.0,1,30,0,0.79,766
4422,2022.0,13.0,9.4,-0.8,153.8,0,0,1,180.0,1,20,0,-3.12,380


In [9]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score 

X = df[['Пол', 'Возраст при зачислении', 'Дневная/вечерняя форма обучения', 
        'Вступительный балл', 'Направление','Предыдущее образование (оценка)']]
y = df['Количество абитуриентов']

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# Параметры кросс-валидации
n_iter = 10  # Количество итераций
test_size = 0.2    # Размер тестовой выборки (20%)
mse_scores = []  # Список для хранения MSE на каждой итерации 
r2_scores = []   # Список для хранения R² на каждой итерации 

for i in range(n_iter): 
    # Случайное разбиение данных на обучающую и тестовую выборки 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i) 

    # Создание и обучение модели (Линейная регрессия) 
    linear_model = LinearRegression() 
    linear_model.fit(X_train, y_train) 

    # Предсказание 
    y_pred = linear_model.predict(X_test) 

    # Оценка производительности модели 
    mse = mean_squared_error(y_test, y_pred)  # Среднеквадратичная ошибка
    r2 = r2_score(y_test, y_pred)              # Коэффициент детерминации

    mse_scores.append(mse) 
    r2_scores.append(r2)

    print(f'Итерация {i + 1}: MSE = {mse:.2f}, R² = {r2:.2f}') 

# Средние значения за все итерации 
mean_mse = np.mean(mse_scores) 
mean_r2 = np.mean(r2_scores)

print(f'Средняя MSE за {n_iter} итераций: {mean_mse:.2f}')
print(f'Средний R² за {n_iter} итераций: {mean_r2:.2f}')

Итерация 1: MSE = 37944.11, R² = 0.09
Итерация 2: MSE = 37211.64, R² = 0.11
Итерация 3: MSE = 37074.41, R² = 0.10
Итерация 4: MSE = 39748.18, R² = 0.11
Итерация 5: MSE = 37846.27, R² = 0.08
Итерация 6: MSE = 38424.71, R² = 0.10
Итерация 7: MSE = 34838.52, R² = 0.10
Итерация 8: MSE = 37589.16, R² = 0.09
Итерация 9: MSE = 36348.58, R² = 0.08
Итерация 10: MSE = 37333.76, R² = 0.07
Средняя MSE за 10 итераций: 37435.93
Средний R² за 10 итераций: 0.09


In [15]:
from sklearn.ensemble import RandomForestClassifier

# Параметры кросс-валидации
n_iter = 10  # Количество итераций
test_size = 0.2    # Размер тестовой выборки (20%)

accuracies_for = []  # Список для хранения точности на каждой итерации

for i in range(n_iter):
    # Случайное разбиение данных на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i)

    # Создание и обучение модели (Random Forest)
    forest_model = RandomForestClassifier(random_state=42)
    forest_model.fit(X_train, y_train)
        
    # Предсказание
    y_pred_for = forest_model.predict(X_test)
        
    # Оценка точности модели
    accuracy = accuracy_score(y_test, y_pred_for)
    accuracies_for.append(accuracy)
    print(f'Итерация {i + 1}: Точность модели = {accuracy:.2f}')

# Средняя точность за все итерации
mean_accuracy = np.mean(accuracies_for)
print(f'Средняя точность за {n_iter} итераций: {mean_accuracy:.2f}')

Итерация 1: Точность модели = 0.98
Итерация 2: Точность модели = 0.98
Итерация 3: Точность модели = 0.98
Итерация 4: Точность модели = 0.99
Итерация 5: Точность модели = 0.98
Итерация 6: Точность модели = 0.99
Итерация 7: Точность модели = 0.98
Итерация 8: Точность модели = 0.98
Итерация 9: Точность модели = 0.98
Итерация 10: Точность модели = 0.98
Средняя точность за 10 итераций: 0.98


In [16]:
from sklearn.tree import DecisionTreeClassifier

# Параметры кросс-валидации
n_iter = 10  # Количество итераций
test_size = 0.2    # Размер тестовой выборки (20%)

accuracies_tree = []  # Список для хранения точности на каждой итерации

for i in range(n_iter):
    # Случайное разбиение данных на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i)

    # Создание и обучение модели
    decision_tree_model = DecisionTreeClassifier(random_state=42)
    decision_tree_model.fit(X_train, y_train)

    # Предсказание 
    y_pred_tree = decision_tree_model.predict(X_test)

    # Оценка точности модели
    accuracy = accuracy_score(y_test, y_pred_tree)
    accuracies_tree.append(accuracy)
    print(f'Итерация {i + 1}: Точность модели = {accuracy:.2f}')

# Средняя точность за все итерации
mean_accuracy = np.mean(accuracies_tree)
print(f'Средняя точность за {n_iter} итераций: {mean_accuracy:.2f}')

Итерация 1: Точность модели = 1.00
Итерация 2: Точность модели = 1.00
Итерация 3: Точность модели = 1.00
Итерация 4: Точность модели = 1.00
Итерация 5: Точность модели = 1.00
Итерация 6: Точность модели = 1.00
Итерация 7: Точность модели = 1.00
Итерация 8: Точность модели = 1.00
Итерация 9: Точность модели = 1.00
Итерация 10: Точность модели = 1.00
Средняя точность за 10 итераций: 1.00


In [19]:
from xgboost import XGBRegressor

# Параметры кросс-валидации 
n_iter = 10  # Количество итераций 
test_size = 0.2  # Размер тестовой выборки (20%) 

mse_scores = []  # Список для хранения MSE на каждой итерации 
r2_scores = []   # Список для хранения R² на каждой итерации 

for i in range(n_iter): 
    # Случайное разбиение данных на обучающую и тестовую выборки 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i) 

    # Создание и обучение модели XGBoost
    xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
    xgb_model.fit(X_train, y_train) 

    # Предсказание 
    y_pred = xgb_model.predict(X_test) 

    # Оценка производительности модели 
    mse = mean_squared_error(y_test, y_pred)  # Среднеквадратичная ошибка
    r2 = r2_score(y_test, y_pred)              # Коэффициент детерминации

    mse_scores.append(mse) 
    r2_scores.append(r2)

    print(f'Итерация {i + 1}: MSE = {mse:.2f}, R² = {r2:.2f}') 

# Средние значения за все итерации 
mean_mse = np.mean(mse_scores) 
mean_r2 = np.mean(r2_scores)

print(f'Средняя MSE за {n_iter} итераций: {mean_mse:.2f}')
print(f'Средний R² за {n_iter} итераций: {mean_r2:.2f}')

Итерация 1: MSE = 0.00, R² = 1.00
Итерация 2: MSE = 0.00, R² = 1.00
Итерация 3: MSE = 0.00, R² = 1.00
Итерация 4: MSE = 0.00, R² = 1.00
Итерация 5: MSE = 0.00, R² = 1.00
Итерация 6: MSE = 0.00, R² = 1.00
Итерация 7: MSE = 0.00, R² = 1.00
Итерация 8: MSE = 0.00, R² = 1.00
Итерация 9: MSE = 0.00, R² = 1.00
Итерация 10: MSE = 0.00, R² = 1.00
Средняя MSE за 10 итераций: 0.00
Средний R² за 10 итераций: 1.00
